# L1: Create Agents to Research and Prepare a Quiz

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

# Install dependencies for Google Colab

In [1]:
!pip install crewai crewai_tools langchain_openai langchain-community langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 13.7 MB/s eta 

# Import Requirements

* Import libraries, APIs and LLM

In [10]:
from crewai import Agent, Task, Crew, LLM
from langchain_openai.chat_models import ChatOpenAI


# crewAI Tools
* [Serper](https://serpapi.com/)

## Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [11]:
from crewai_tools import SerperDevTool

# Setup Groq and Serper API Keys

## Setting Up Groq API Key

Groq provides access to AI models via its API. Follow these steps to get your API key:

### Steps:
1. Visit [Groq API Dashboard](https://console.groq.com/).

2. Sign in or Sign up

3. Generate API Key
   - Navigate to the **API Keys** section.
   - Click **Create New Key**.
   - Copy and securely store the generated key.


## Setting Up Serper API Key
Serper is a search API that allows programmatic access to Google search results.

### Steps:
1. Visit [Serper API Portal](https://serpapi.com/)
2. Sign in or Sign up
3. Register if you are a new user. Otherwise, log in with your credentials.
4. Generate API Key
   - Click Generate API Key.
   - Copy and securely store the API key.

In [18]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["SERP_API_KEY"] = userdata.get("SERP_API_KEY")



In [28]:
llm = ChatOpenAI(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.deepseek.com/v1",
    temperature=0.2
)

llm = LLM(model="groq/llama-3.3-70b-versatile")

# Create Agents

In [31]:
# Initialize tools
search_tool = SerperDevTool()

# Start Working on your Tool

In [39]:
# fetch latest stock/crypto prices and news
data_fetch_agent = Agent(
    role="Data Fetch Agent",
    goal="Retrieve stock/crypto latest news.",
    backstory=(
        "A seasoned financial researcher with years of experience in market analysis. "
        "Has a keen eye for reliable sources and knows how to extract the most "
        "accurate and relevant data in real time."
    ),
    tools=[search_tool],
    verbose=True,
    llm=llm
)

# perform sentiment analysis to extract current market situation
sentiment_analysis_agent = Agent(
    role="Sentiment Analysis Expert",
    goal="Analyze sentiment of stock/crypto news.",
    backstory=(
        "A former hedge fund analyst turned AI-driven sentiment expert. "
        "Skilled in deciphering news trends and understanding how market sentiment "
        "influences stock movements."
    ),
    tools=[],
    verbose=True,
    llm=llm
)

# Based on the analysis, provides decision of buying or selling the stock
decision_agent = Agent(
    role="Trading Decision Specialist",
    goal="Evaluate sentiment and provide a trading decision.",
    backstory=(
        "A data-driven financial strategist with experience in algorithmic trading. "
        "Uses both quantitative and qualitative insights to make precise and "
        "calculated trading decisions."
    ),
    tools=[],
    verbose=True,
    llm=llm

)

# Define Tasks

In [40]:
# Research Task
data_fetch_task = Task(
    description=(
        "Gather the latest news from reliable financial sources regarding {topic}. "
        "Ensure sources include reputable financial news websites, market analysis reports, "
        "and expert opinions. Prioritize news from the last 24 hours to maintain relevance."
    ),
    expected_output=(
        "Curated list of latest financial news with:\n"
        "- Headline\n"
        "- Source URL\n"
        "- Publication date\n"
        "- Summary of key insights\n"
    ),
    agent=data_fetch_agent
)


sentiment_analysis_task = Task(
    description=(
        "Assess the latest financial news and market updates from the fetched data. "
        "Analyze the overall business impact of recent events, such as new product launches, "
        "market competition, regulatory changes, financial reports, and other key developments. "
        "Determine whether the current news suggests a potential growth, risk, or stability phase for the company."
    ),
    expected_output=(
        "Business impact analysis report with:\n"
        "- Summary of key news headlines\n"
        "- Identified business factors (e.g., competition, innovation, regulations, earnings reports)\n"
        "- Assessment of how these factors influence the company's growth or decline\n"
        "- Risk assessment (e.g., threats from competitors, legal issues, supply chain problems)\n"
        "- Overall market perception: Is the company in a phase of growth, risk, or stability?"
    ),
    agent=sentiment_analysis_agent,
    context=[data_fetch_task]
)


trading_decision_task = Task(
    description=(
        "Based on the business impact assessment of {topic}, "
        "determine the best trading strategy. Consider:\n"
        "- Growth phase: If the company shows innovation, strong financials, or market expansion, suggest BUY.\n"
        "- Risk phase: If the company faces serious threats like legal issues, strong competition, or declining revenue, suggest SELL.\n"
        "- Stability phase: If there are no major shifts, suggest HOLD.\n"
        "Ensure that the recommendation aligns with market trends and fundamental analysis."
    ),
    expected_output=(
        "Trading recommendation report with:\n"
        "- Summary of sentiment analysis findings\n"
        "- Clear BUY, SELL, or HOLD recommendation\n"
        "- Justification based on market conditions, financials, and recent events\n"
        "- Optional risk factors or alternate scenarios to consider"
    ),
    agent=decision_agent,
    context=[sentiment_analysis_task]
)


# Crew

In [41]:
trading_partner = Crew(
    agents=[data_fetch_agent, sentiment_analysis_agent, decision_agent],
    tasks=[data_fetch_task, sentiment_analysis_task, trading_decision_task]
)

In [42]:
result = trading_partner.kickoff(inputs={'topic': "Apple"})
print(result)

# Agent: Data Fetch Agent
## Task: Gather the latest news from reliable financial sources regarding Apple. Ensure sources include reputable financial news websites, market analysis reports, and expert opinions. Prioritize news from the last 24 hours to maintain relevance.
 

I encountered an error while trying to use the tool. This was the error: 'SERPER_API_KEY'.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Data Fetch Agent
## Thought: Thought: To gather the latest news from reliable financial sources regarding Apple, I should use the internet search tool with a specific query that targets financial news and recent publication dates. This will h

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jj2fjhrxe84tdb623db21n6f` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 6238, Requested 1415. Please try again in 16.538s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}



 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jj2fjhrxe84tdb623db21n6f` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 6238, Requested 1415. Please try again in 16.538s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jj2fjhrxe84tdb623db21n6f` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 6238, Requested 1415. Please try again in 16.538s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}
